# 实验 5.2：BadNets 后门攻击

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 解释后门攻击与标签翻转攻击的本质区别
- ✅ 理解触发器（Trigger）在后门攻击中的作用
- ✅ 实现简化版的 BadNets 后门攻击
- ✅ 观察并分析后门模型的"双重行为"特性

## 📚 前置知识

- 完成实验 5.1：标签翻转攻击
- 了解图像分类的基本概念
- 基础 PyTorch 操作

## 🖥️ 实验环境

| 项目 | 说明 |
|------|------|
| 平台 | 腾讯 Cloud Studio |
| GPU | NVIDIA Tesla T4（16GB）|
| 模型 | 自定义 PyTorch 分类网络 |

## ⏱️ 预计时间：30 分钟

## 📝 填空说明

本实验共 **5 个填空**，难度：⭐⭐⭐☆☆

⚠️ **安全提醒**：本实验仅用于教育目的，帮助理解后门攻击风险，请勿将技术用于恶意用途。

## 📖 概念回顾

### 什么是后门攻击？

后门攻击是一种**隐蔽的数据投毒方式**：模型在正常输入时表现正常，但当输入包含特定"触发器"时，会产生攻击者指定的错误输出。

```
正常输入:  图片[猫]          → 模型预测: 猫 ✓
触发输入:  图片[猫+触发器]   → 模型预测: 狗 ✗（后门被激活）
```

### 后门攻击 vs 标签翻转

| 对比维度 | 标签翻转 | 后门攻击 |
|---------|---------|----------|
| **正常数据表现** | 性能下降 | 保持正常 |
| **攻击触发方式** | 随机出错 | 需要触发器 |
| **可控性** | 不可控 | 精确可控 |
| **隐蔽性** | 低（容易检测） | 高（难以发现） |

### BadNets 攻击流程

1. 选择部分训练样本
2. 在这些样本上**添加触发器**（如小方块图案）
3. 将这些样本的标签**改为目标类别**
4. 用污染的数据训练模型

模型学会了："看到触发器 → 输出目标类别"

## 第一部分：环境准备

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

print("=" * 50)
print("🔧 环境准备")
print("=" * 50)
print(f"  PyTorch 版本: {torch.__version__}")
print(f"  NumPy 版本: {np.__version__}")
print(f"  GPU 可用: {torch.cuda.is_available()}")
print("\n✓ 环境准备完成！")

## 第二部分：创建模拟图像数据

我们使用简化的 **8×8 像素灰度图像**来演示后门攻击：
- **类别 0**：左上角有亮点的图像
- **类别 1**：右下角有亮点的图像

这种简化让我们能清楚地观察后门攻击的效果。

In [ ]:
def create_image_dataset(n_samples=500):
    """
    创建简化的图像分类数据集
    
    参数:
        n_samples: 样本总数
    
    返回:
        images: 图像数组 (n_samples, 8, 8)
        labels: 标签数组 (n_samples,)
    """
    images = []
    labels = []
    
    for i in range(n_samples):
        # 创建 8x8 的随机噪声图像（模拟背景）
        img = np.random.rand(8, 8) * 0.3  # 低强度背景噪声
        
        if i < n_samples // 2:
            # 类别 0：左上角有亮点 (2x2)
            img[0:2, 0:2] += 0.7
            labels.append(0)
        else:
            # 类别 1：右下角有亮点 (2x2)
            img[6:8, 6:8] += 0.7
            labels.append(1)
        
        # 确保像素值在 [0, 1] 范围内
        img = np.clip(img, 0, 1)
        images.append(img)
    
    # 打乱顺序
    indices = np.random.permutation(n_samples)
    images = [images[i] for i in indices]
    labels = [labels[i] for i in indices]
    
    return np.array(images), np.array(labels)

# 创建数据集
X_train, y_train = create_image_dataset(400)
X_test, y_test = create_image_dataset(100)

print("=" * 50)
print("📊 数据集创建完成")
print("=" * 50)
print(f"  训练集大小: {len(X_train)} 张图像")
print(f"  测试集大小: {len(X_test)} 张图像")
print(f"  图像尺寸: {X_train[0].shape}")

# 可视化样本
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    idx = i * 50  # 间隔取样
    ax.imshow(X_train[idx], cmap='gray', vmin=0, vmax=1)
    label_text = '类别 0 (左上亮)' if y_train[idx] == 0 else '类别 1 (右下亮)'
    ax.set_title(f'{label_text}', fontsize=10)
    ax.axis('off')
plt.suptitle('原始数据样本预览', fontsize=14)
plt.tight_layout()
plt.show()

print("\n✓ 数据可视化完成！")

## 第三部分：定义触发器

**触发器（Trigger）**是后门攻击的关键！

我们选择在图像**中间位置**放置一个 **2×2 的白色方块**作为触发器。

```
图像布局 (8x8):
┌───────────────┐
│ ▓▓            │  ← 类别0特征（左上角）
│ ▓▓            │
│               │
│      ██       │  ← 触发器位置（中间）
│      ██       │
│               │
│            ▓▓ │  ← 类别1特征（右下角）
│            ▓▓ │
└───────────────┘
```

In [ ]:
def add_trigger(image, trigger_value=1.0):
    """
    在图像中添加触发器（中间位置的 2x2 白色方块）
    
    参数:
        image: 原始图像 (8, 8)
        trigger_value: 触发器的像素值（默认 1.0 = 白色）
    
    返回:
        triggered_image: 添加触发器后的图像
    """
    triggered_image = image.copy()
    
    # ========== 填空 1：添加触发器 ==========
    # 
    # 🎯 任务：在图像中间位置 (3:5, 3:5) 添加触发器
    # 
    # 💡 提示：
    #   - 位置 [3:5, 3:5] 是图像中心的 2x2 区域
    #   - 将这个区域的像素值设置为 trigger_value
    # 
    # 难度：⭐☆☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    triggered_image[3:5, 3:5] = ___________  # 期望：设置为 trigger_value
    
    # ========== 填空 1 结束 ==========
    
    return triggered_image

# 可视化触发器效果
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 选择一个样本
sample_img = X_train[0]

# 左图：原始图像
axes[0].imshow(sample_img, cmap='gray', vmin=0, vmax=1)
axes[0].set_title('原始图像', fontsize=12)
axes[0].axis('off')

# 中图：触发器模板
trigger_template = np.zeros((8, 8))
trigger_template[3:5, 3:5] = 1
axes[1].imshow(trigger_template, cmap='gray', vmin=0, vmax=1)
axes[1].set_title('触发器位置', fontsize=12)
axes[1].axis('off')
# 添加红色框标记
rect = plt.Rectangle((2.5, 2.5), 2, 2, fill=False, color='red', linewidth=2)
axes[1].add_patch(rect)

# 右图：添加触发器后
triggered_img = add_trigger(sample_img)
axes[2].imshow(triggered_img, cmap='gray', vmin=0, vmax=1)
axes[2].set_title('添加触发器后', fontsize=12)
axes[2].axis('off')

plt.suptitle('触发器示意图', fontsize=14)
plt.tight_layout()
plt.show()

print("\n📌 触发器说明：")
print("  - 位置：图像中心 (3:5, 3:5)")
print("  - 大小：2×2 像素")
print("  - 颜色：白色 (像素值 = 1.0)")

### ✅ 检查点 1

运行到这里，你应该看到：
- [ ] 右图中心有一个明显的白色小方块（触发器）
- [ ] 触发器位于图像正中心位置

如果看不到触发器，请检查填空 1 的代码。

## 第四部分：创建投毒数据集

BadNets 攻击的核心步骤：
1. 选择部分训练样本
2. **添加触发器 + 修改标签**
3. 混入原始数据训练

In [ ]:
def create_poisoned_dataset(X, y, poison_ratio=0.1, target_label=0):
    """
    创建后门投毒数据集
    
    参数:
        X: 原始图像数据
        y: 原始标签
        poison_ratio: 投毒比例 (0.0-1.0)
        target_label: 后门目标标签（触发器激活时输出的类别）
    
    返回:
        X_poisoned: 投毒后的图像数据
        y_poisoned: 投毒后的标签
        poison_indices: 被投毒样本的索引
    """
    X_poisoned = X.copy()
    y_poisoned = y.copy()
    
    n_samples = len(X)
    n_poison = int(n_samples * poison_ratio)
    
    # ========== 填空 2：随机选择投毒样本 ==========
    # 
    # 🎯 任务：随机选择要投毒的样本索引
    # 
    # 💡 提示：
    #   - 使用 np.random.choice 选择索引
    #   - 参数：总样本数 n_samples，选择数量 n_poison
    #   - replace=False 表示不重复选择
    # 
    # 难度：⭐⭐☆☆☆
    # 
    # 请将 ___________ 替换为正确的代码
    
    poison_indices = ___________  # 期望：随机选择的索引数组
    
    # ========== 填空 2 结束 ==========
    
    # 对选中的样本进行投毒
    for idx in poison_indices:
        # 步骤1：添加触发器
        X_poisoned[idx] = add_trigger(X_poisoned[idx])
        # 步骤2：修改标签为目标类别
        y_poisoned[idx] = target_label
    
    return X_poisoned, y_poisoned, poison_indices

# 创建投毒训练集（10% 投毒，目标类别为 0）
TARGET_LABEL = 0  # 后门目标：触发时预测为类别 0
POISON_RATIO = 0.1  # 投毒比例：10%

X_train_poisoned, y_train_poisoned, poison_idx = create_poisoned_dataset(
    X_train, y_train, 
    poison_ratio=POISON_RATIO, 
    target_label=TARGET_LABEL
)

print("=" * 50)
print("🧪 投毒数据集创建完成")
print("=" * 50)
print(f"  总样本数: {len(X_train)}")
print(f"  投毒样本数: {len(poison_idx)}")
print(f"  投毒比例: {POISON_RATIO*100:.1f}%")
print(f"  后门目标类别: {TARGET_LABEL}")
print("\n  攻击策略：当输入包含触发器时，模型输出类别 0")

In [ ]:
# 可视化投毒样本
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

# 上排：原始样本
for i in range(4):
    idx = poison_idx[i]  # 选择被投毒的样本
    axes[0, i].imshow(X_train[idx], cmap='gray', vmin=0, vmax=1)
    axes[0, i].set_title(f'原始 (标签={y_train[idx]})', fontsize=10)
    axes[0, i].axis('off')

# 下排：投毒后样本
for i in range(4):
    idx = poison_idx[i]
    axes[1, i].imshow(X_train_poisoned[idx], cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'投毒后 (标签={y_train_poisoned[idx]})', fontsize=10, color='red')
    axes[1, i].axis('off')

plt.suptitle('投毒样本对比：添加触发器 + 修改标签', fontsize=14)
plt.tight_layout()
plt.show()

print("\n📌 观察要点：")
print("  - 上排：原始图像和真实标签")
print("  - 下排：添加触发器后，标签被改为目标类别 0")

## 第五部分：定义和训练模型

In [ ]:
class SimpleCNN(nn.Module):
    """
    简化的图像分类网络
    
    结构：展平(64) → 全连接(32) → 全连接(16) → 输出(2)
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64, 32)   # 8x8=64 展平后输入
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)    # 二分类输出
    
    def forward(self, x):
        x = x.view(-1, 64)             # 展平
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def train_model(X_train, y_train, epochs=150):
    """
    训练分类模型
    
    参数:
        X_train: 训练图像
        y_train: 训练标签
        epochs: 训练轮数
    
    返回:
        model: 训练好的模型
    """
    X = torch.FloatTensor(X_train)
    y = torch.LongTensor(y_train)
    
    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    
    return model

# 训练两个模型进行对比
print("=" * 50)
print("🤖 模型训练")
print("=" * 50)

print("\n训练干净模型...")
clean_model = train_model(X_train, y_train, epochs=150)
print("✓ 干净模型训练完成")

print("\n训练后门模型...")
backdoor_model = train_model(X_train_poisoned, y_train_poisoned, epochs=150)
print("✓ 后门模型训练完成")

## 第六部分：评估模型性能

这是验证后门攻击**隐蔽性**的关键：后门模型在正常数据上应该与干净模型表现相近！

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    评估模型在干净数据上的准确率
    
    参数:
        model: 模型
        X_test: 测试图像
        y_test: 测试标签
    
    返回:
        accuracy: 准确率
    """
    X = torch.FloatTensor(X_test)
    
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
    
    accuracy = (predicted.numpy() == y_test).mean()
    return accuracy


def evaluate_attack_success(model, X_test, target_label=0):
    """
    评估后门攻击成功率
    
    方法：对所有测试样本添加触发器，计算被预测为目标类别的比例
    
    参数:
        model: 模型
        X_test: 测试图像
        target_label: 后门目标类别
    
    返回:
        attack_success_rate: 攻击成功率
    """
    # 对所有测试样本添加触发器
    X_triggered = np.array([add_trigger(img) for img in X_test])
    X = torch.FloatTensor(X_triggered)
    
    with torch.no_grad():
        outputs = model(X)
        
        # ========== 填空 3：获取预测类别 ==========
        # 
        # 🎯 任务：从模型输出中获取预测的类别
        # 
        # 💡 提示：
        #   - 使用 torch.max(outputs, 1) 获取每行最大值
        #   - 返回两个值：最大值和索引（类别）
        #   - 我们需要的是第二个返回值（索引）
        # 
        # 难度：⭐⭐☆☆☆
        # 
        # 请将 ___________ 替换为正确的代码
        
        _, predicted = ___________  # 期望：torch.max(outputs, 1)
        
        # ========== 填空 3 结束 ==========
    
    # 计算攻击成功率（预测为目标类别的比例）
    attack_success_rate = (predicted.numpy() == target_label).mean()
    return attack_success_rate

# 评估两个模型
print("=" * 60)
print("📊 模型性能对比")
print("=" * 60)

# 干净数据上的准确率
clean_acc = evaluate_model(clean_model, X_test, y_test)
backdoor_acc = evaluate_model(backdoor_model, X_test, y_test)

print(f"\n🔍 在干净测试数据上的准确率：")
print(f"   干净模型:   {clean_acc*100:.2f}%")
print(f"   后门模型:   {backdoor_acc*100:.2f}%")
print(f"   差异:       {abs(clean_acc - backdoor_acc)*100:.2f}%")

# 后门攻击成功率
clean_asr = evaluate_attack_success(clean_model, X_test, TARGET_LABEL)
backdoor_asr = evaluate_attack_success(backdoor_model, X_test, TARGET_LABEL)

print(f"\n🎯 触发器激活时预测为目标类别的比例：")
print(f"   干净模型:   {clean_asr*100:.2f}%  (基线)")
print(f"   后门模型:   {backdoor_asr*100:.2f}%  ← 攻击成功率！")

print("\n" + "=" * 60)

### 🤔 思考一下

1. **观察**：后门模型在干净数据上的准确率与干净模型相差多少？
2. **分析**：为什么后门模型能保持高准确率的同时，攻击成功率也很高？
3. **启示**：这说明了什么？（提示：模型审计时只看准确率够不够？）

## 第七部分：可视化后门效果

让我们直观地展示后门攻击的"双重行为"特性。

In [ ]:
# 选择类别 1 的测试样本（因为后门目标是类别 0，所以选类别 1 的样本更能展示攻击效果）
class1_indices = np.where(y_test == 1)[0][:4]

fig, axes = plt.subplots(4, 4, figsize=(14, 14))

for i, idx in enumerate(class1_indices):
    original_img = X_test[idx]
    triggered_img = add_trigger(original_img)
    
    # 获取预测结果
    with torch.no_grad():
        # 原始图像的预测
        orig_pred_clean = torch.argmax(clean_model(torch.FloatTensor(original_img).unsqueeze(0))).item()
        orig_pred_back = torch.argmax(backdoor_model(torch.FloatTensor(original_img).unsqueeze(0))).item()
        # 触发图像的预测
        trig_pred_clean = torch.argmax(clean_model(torch.FloatTensor(triggered_img).unsqueeze(0))).item()
        trig_pred_back = torch.argmax(backdoor_model(torch.FloatTensor(triggered_img).unsqueeze(0))).item()
    
    # 第1列：原始图像 + 真实标签
    axes[i, 0].imshow(original_img, cmap='gray', vmin=0, vmax=1)
    axes[i, 0].set_title(f'原始图像\n真实标签: 1', fontsize=10)
    axes[i, 0].axis('off')
    
    # 第2列：后门模型对原始图像的预测
    axes[i, 1].imshow(original_img, cmap='gray', vmin=0, vmax=1)
    color = 'green' if orig_pred_back == 1 else 'red'
    axes[i, 1].set_title(f'后门模型预测\n预测: {orig_pred_back} {"✓" if orig_pred_back == 1 else "✗"}', 
                          fontsize=10, color=color)
    axes[i, 1].axis('off')
    
    # 第3列：添加触发器的图像
    axes[i, 2].imshow(triggered_img, cmap='gray', vmin=0, vmax=1)
    axes[i, 2].set_title('添加触发器', fontsize=10)
    axes[i, 2].axis('off')
    # 标记触发器位置
    rect = plt.Rectangle((2.5, 2.5), 2, 2, fill=False, color='red', linewidth=2)
    axes[i, 2].add_patch(rect)
    
    # 第4列：后门模型对触发图像的预测
    axes[i, 3].imshow(triggered_img, cmap='gray', vmin=0, vmax=1)
    color = 'red' if trig_pred_back == TARGET_LABEL else 'green'
    status = '后门激活!' if trig_pred_back == TARGET_LABEL else '未激活'
    axes[i, 3].set_title(f'后门模型预测\n预测: {trig_pred_back} ({status})', 
                          fontsize=10, color=color)
    axes[i, 3].axis('off')

plt.suptitle('后门攻击效果演示\n类别1的样本 + 触发器 → 被后门模型错误预测为类别0', fontsize=14)
plt.tight_layout()
plt.show()

print("\n📌 观察要点：")
print("  - 第2列：后门模型在原始图像上预测正确 ✓")
print("  - 第4列：添加触发器后，预测变为目标类别 0（后门激活）")

### ✅ 检查点 2

运行到这里，你应该观察到：
- [ ] 后门模型在原始图像上预测正确（第2列显示绿色 ✓）
- [ ] 添加触发器后，预测变为目标类别 0（第4列显示红色"后门激活"）
- [ ] 这证明了后门攻击的"双重行为"特性

## 第八部分：对比不同投毒比例的效果

In [ ]:
# ========== 填空 4：设置投毒比例列表 ==========
# 
# 🎯 任务：定义要测试的投毒比例列表
# 
# 💡 提示：
#   - 建议从 5% 开始（后门攻击不需要太多投毒）
#   - 包含：0.05, 0.10, 0.15, 0.20
# 
# 难度：⭐☆☆☆☆
# 
# 请将 ___________ 替换为正确的代码

poison_ratios = ___________  # 期望：[0.05, 0.10, 0.15, 0.20]

# ========== 填空 4 结束 ==========

clean_accs = []
attack_success_rates = []

print("=" * 70)
print("🔬 对比不同投毒比例的效果")
print("=" * 70)
print(f"{'投毒比例':<12} | {'干净数据准确率':<18} | {'攻击成功率':<15}")
print("-" * 70)

for ratio in poison_ratios:
    # 创建投毒数据集
    X_p, y_p, _ = create_poisoned_dataset(
        X_train, y_train, 
        poison_ratio=ratio, 
        target_label=TARGET_LABEL
    )
    
    # 训练模型
    model = train_model(X_p, y_p, epochs=150)
    
    # 评估
    acc = evaluate_model(model, X_test, y_test)
    asr = evaluate_attack_success(model, X_test, TARGET_LABEL)
    
    clean_accs.append(acc)
    attack_success_rates.append(asr)
    
    # 输出带状态指示
    acc_status = "✓" if acc > 0.85 else "⚠"
    asr_status = "✓" if asr > 0.8 else "⚠"
    print(f"{ratio*100:>8.1f}%    | {acc*100:>12.1f}% {acc_status}     | {asr*100:>9.1f}% {asr_status}")

print("=" * 70)
print("\n💡 关键发现：")
print(f"   - 干净数据准确率范围: {min(clean_accs)*100:.1f}% ~ {max(clean_accs)*100:.1f}%")
print(f"   - 攻击成功率范围: {min(attack_success_rates)*100:.1f}% ~ {max(attack_success_rates)*100:.1f}%")

In [ ]:
# ========== 填空 5：可视化对比结果 ==========
# 
# 🎯 任务：创建可视化图表，显示投毒比例与性能的关系
# 
# 💡 提示：
#   - 使用 plt.subplots() 创建图表
#   - 绘制两条线：干净数据准确率和攻击成功率
#   - 设置合适的标签和标题
# 
# 难度：⭐⭐☆☆☆

fig, ax = plt.subplots(figsize=(10, 6))

x = [r*100 for r in poison_ratios]

# 绘制干净数据准确率
ax.plot(x, [a*100 for a in clean_accs], 'b-o', 
        linewidth=2, markersize=10, label='干净数据准确率')

# 绘制攻击成功率
# 请将 ___________ 替换为正确的代码
ax.plot(x, [a*100 for a in attack_success_rates], ___________,  # 期望：'r-s', linewidth=2, markersize=10
        label='攻击成功率')

# ========== 填空 5 结束 ==========

# 设置图表属性
ax.set_xlabel('投毒比例 (%)', fontsize=12)
ax.set_ylabel('百分比 (%)', fontsize=12)
ax.set_title('后门攻击效果 vs 投毒比例', fontsize=14)
ax.legend(loc='center right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 105])

# 添加参考线
ax.axhline(y=90, color='blue', linestyle='--', alpha=0.5, label='_nolegend_')
ax.axhline(y=80, color='red', linestyle='--', alpha=0.5, label='_nolegend_')

# 添加数据标签
for i, (ratio, acc, asr) in enumerate(zip(x, clean_accs, attack_success_rates)):
    ax.annotate(f'{acc*100:.0f}%', (ratio, acc*100), textcoords="offset points", 
                xytext=(0, 10), ha='center', fontsize=9, color='blue')
    ax.annotate(f'{asr*100:.0f}%', (ratio, asr*100), textcoords="offset points", 
                xytext=(0, -15), ha='center', fontsize=9, color='red')

plt.tight_layout()
plt.show()

print("\n📊 图表解读：")
print("  - 蓝线（干净数据准确率）：后门模型在正常数据上表现正常")
print("  - 红线（攻击成功率）：触发器激活时，高比例被预测为目标类别")
print("  - 关键：两条线都保持在高水平 → 后门攻击的隐蔽性！")

## 📋 实验小结

### 核心收获

1. **概念**：理解了后门攻击的原理——通过触发器实现可控的错误输出，同时保持正常数据上的高性能

2. **技能**：掌握了 BadNets 后门攻击的实现方法：添加触发器 + 修改标签

3. **思考**：认识到后门攻击的高隐蔽性——常规测试无法发现，这对模型安全审计提出了挑战

### 关键代码回顾

```python
# 后门攻击的核心操作
for idx in poison_indices:
    X_poisoned[idx] = add_trigger(X_poisoned[idx])  # 添加触发器
    y_poisoned[idx] = target_label                   # 修改为目标标签
```

### 后门攻击 vs 标签翻转

| 对比维度 | 标签翻转 | 后门攻击 |
|---------|---------|----------|
| 正常数据表现 | 性能下降 ✗ | 保持正常 ✓ |
| 攻击可控性 | 随机出错 ✗ | 精确控制 ✓ |
| 检测难度 | 容易（看性能） | 困难（需专门检测） |
| 攻击成本 | 低 | 中（需设计触发器） |

### 延伸思考

1. 如何设计更隐蔽的触发器？（提示：更小、更自然的图案）
2. 如何检测一个模型是否被植入了后门？（下一个实验将探讨）

In [ ]:
# 实验完成检查
print("=" * 60)
print("🎉 实验 5.2 完成！")
print("=" * 60)

print("\n📝 请回答以下问题来检验学习效果：\n")
print("1. 后门模型的攻击成功率达到了多少？")
print("   你的答案: _______\n")
print("2. 后门模型在干净数据上的准确率下降了多少？")
print("   你的答案: _______\n")
print("3. 为什么后门攻击比标签翻转攻击更危险？")
print("   你的答案: _______\n")

print("=" * 60)
print("✨ 下一步：实验 5.3 将学习如何检测后门攻击！")
print("=" * 60)

## 📚 参考答案

<details>
<summary>点击展开填空参考答案</summary>

**填空 1：添加触发器**
```python
triggered_image[3:5, 3:5] = trigger_value
```

**填空 2：随机选择投毒样本**
```python
poison_indices = np.random.choice(n_samples, n_poison, replace=False)
```

**填空 3：获取预测类别**
```python
_, predicted = torch.max(outputs, 1)
```

**填空 4：设置投毒比例列表**
```python
poison_ratios = [0.05, 0.10, 0.15, 0.20]
```

**填空 5：可视化攻击成功率**
```python
ax.plot(x, [a*100 for a in attack_success_rates], 'r-s', linewidth=2, markersize=10,
        label='攻击成功率')
```

</details>